### Data Generation

In [ ]:
import random as rd
import pandas as pd

###innitiate values that are shuffled around for more varied prompts and sql queries

template=pd.read_excel(r"C:\...\templateText.xlsx")#here the code accesses the template containing the natural language phrases
templatesql=pd.read_excel(r"C:\...\templateSQL.xlsx")#here the code accessed the template with the corresponding SQL commands
countryList=pd.read_excel(r"C:\...\countries.xlsx")#here the list of the countries used is accessed
listdecide = ['اقل من', "اكثر من", "اعلى من", "ارخص من", "اغلى من", "يقارب"]#here the types of compares are accessed (less than, more than...)
listdecidesql = ['< X4', '> X4', '\t > X4', '\t < X4', '\t > X4', 'BETWEEN 50 + X4 AND X4 -50']#here are the corresponding comparsions
X6Text = "في يوم X6"#here is the general format for date in the nl prompt. Placing it here increases the variaty of prompts generated

###innitiate data holders
newdata = pd.DataFrame()
newsql=pd.DataFrame()
full =[]#used for testing, debugging, and length control



while (len(full)  <700):#number of elements
 X6Text = " في يوم X6"
 date = "'"+str(rd.randint(1,28)) + '-' + str(rd.randint(1,12)) + '-' + str(rd.randint(2021,2023))+"'"#select random date
 X6Sql = "AND 'اليوم'=="+date+" "#create the argument to find date
 X6Text = X6Text.replace('X6', date)#create the text argument to add to the prompt

 priceindex = rd.randint(0,len(listdecide)-1) #needed to select the comparsion type
 location1 = countryList.loc[rd.randint(0,len(countryList)-1),:].item()#extract destination
 location2 = countryList.loc[rd.randint(0,len(countryList)-1),:].item()#extract starting location
 price = str(rd.randint(100,500))#range of price depends on data.
 pricechoice = listdecide[priceindex]#decide on what type of price comparsion
 pricechoicesql = listdecidesql[priceindex]#needed for the "between" argument
 degree = str(rd.randint(1,4))#seat degree (1st class, 2nd class, etc...)

 deciderDate = rd.randint(1,100)/100 #the probability of including a date in the final argument



 while location1 == location2:#checks that destination and start are not the same
    location2 = countryList.loc[rd.randint(0,len(countryList)-1),:].item()#look for another location if two are equal
 
 randomindex = rd.randint(0,len(template)-1)#shuffles and selects "base phrase" from the template at random
 sqls =  templatesql.iloc[randomindex,:]
 sqls = sqls.str.replace('Y2' ,  "'البداية'")#to
 sqls = sqls.str.replace('Y1', "'الوجهة'")#from
 sqls = sqls.str.replace('Y3 ', "'التكلفة'") #price in dollars
 sqls = sqls.str.replace('Y5', "'الدرجة'") #price in dollars
 
 sqls= sqls.str.replace('X1' ,  "'"+location1+"'")#to
 sqls = sqls.str.replace('X2', "'"+location2+"'")#from
 sqls = sqls.str.replace('X3', pricechoicesql) #cheaper/> expensive
 sqls = sqls.str.replace('X4', price) #price in dollars
 sqls = sqls.str.replace('X5', "'"+degree+"'")
 
 


 Item =  template.iloc[randomindex,:]
 Item= Item.str.replace('X1' ,  location1)#to
 Item = Item.str.replace('X2', location2)#from
 Item = Item.str.replace('X4', price) #price in dollars
 Item = Item.str.replace('X3', pricechoice) #cheaper/> expensive
 Item = Item.str.replace('X5',"'"+degree+"'")

 #above replaces all value holders
 
 Itemstr = Item.item()
 if deciderDate > 0.5:#if the random gives a value higher than 50%, the line will include date
    sqls = sqls.astype('str')+X6Sql#adds date to sql
    Item = Item.str.replace("؟",X6Text+"؟")#adds date to text
   
 
 
 try:
    if not newdata[0].str.contains(Item.item()).any():
     
     full.append(Itemstr) 
     newdata = pd.concat([newdata,Item])
     newsql = pd.concat([newsql,sqls])
     

 except:
    full.append(Itemstr)
    newdata = pd.concat([newdata,Item])
    newsql = pd.concat([newsql,sqls])

dt = pd.concat([newdata,newsql], axis = 1)#merges all text and sql to a single file
dt.to_excel(r"C:\...\full.xlsx", index = False)#save file





### Prepare Data

In [ ]:
from dotenv import load_dotenv
import openai
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")#extract key from .env file
!python openai tools fine_tunes.prepare_data -f "full.xlsx"#prepare the data. note the headers in full were changed to 'prompt' and 'completion' prior

In [ ]:
#make the training and validation set where the validation is 25%


shuffler = pd.read_json('full_prepared.jsonl', lines=True)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    shuffler['prompt'], shuffler['completion'], random_state=48, test_size=0.25)

Train= pd.concat([X_train, y_train], axis=1)
Validate = pd.concat([X_val, y_val], axis=1)
Train.to_excel(r"C:\....\Full_no_stop_removed_training.xlsx",index=False)#always saved in excel due to decoding issues
Validate.to_excel(r"C:\...\Full_no_stop_removed_validate.xlsx",index=False)
#note the absence of the testing set. It was made using a more varied template to enclude new keywords the program hasn't seen before
#that was done to ensure the avoidance of overfitting. It was then prepared as above

In [ ]:
from nltk.corpus import stopwords
import pandas as pd
stop = stopwords.words('arabic')#stopword extraction


training = pd.read_json(r"C:\....\Full_no_stop_removed_training.jsonl", lines=True)#prepared using fine_tunes.prepare as above
validation = pd.read_json(r"C:\...\Full_no_stop_removed_validate.jsonl", lines=True)

training['prompt'] = training['prompt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
validation['prompt'] = validation['prompt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))#stopwords removal
training.to_excel(r"C:\....\Full_stop_removed_training.xlsx",index=False)#always saved in excel due to decoding issues
validation.to_excel(r"C:\...\Full_stop_removed_validate.xlsx",index=False)

#again, everything was run through the preparer. this was for caution: to tripple check that everything was in the correct format
#because the modules are time-consuming

### Model Creation

In [ ]:
#file upload

tra=openai.File.create(file=open(r"C:\...\Full_no_stop_removed_training_prepared.jsonl", "rb"),purpose='fine-tune')
val=openai.File.create(file=open(r"C:\...\Full_no_stop_removed_validate_prepared.jsonl", "rb"),purpose='fine-tune')
print(tra)
print(val)#look for the ID of the file for each

In [ ]:
openai.FineTune.create(training_file="file-idNum", validation_file = "file-idNum",n_epochs = 5)
#create the module, and retrieve the module ID

In [ ]:
!openai api fine_tunes.follow -i "ft-idModule" #check the status and retrieve module name
#repeat for the files with stopwords removed

### Prediction Collection

In [1]:
completions = [0]*300#placed in a different cell as to only access when starting a new set (to keep the runtime values)

In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
import pandas as pd
df = pd.read_json(r"C:\...\test_prepared.jsonl", lines=True)

from time import sleep
for i in range(0,300):
  response = openai.Completion.create(
    model="modelName",
    prompt=df.iloc[i,:].get('prompt') ,
    temperature=0.3,
    max_tokens=200,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["#", ";","->",'\n']
  )
  completions[i] = response['choices'][0]['text']
  if (i%30==0):
    sleep(60)
    
  


In [ ]:
df['completion'] = completions#cast the results to a dataset when complete

In [ ]:
df.to_json('test complete no stop removed.json')
df.to_excel('test complete no stop removed.xlsx')#save output. Repeat for model recovered from training set with removed stopwords

### Quality Measurment

In [ ]:
from rouge import Rouge
import pandas as pd

rouge = Rouge()

model_in = pd.read_excel('no stop.xlsx')#manually checked and reformatted from previous step to keep only the part we want to measure in the excel
refr=  pd.read_excel('test.xlsx')#likewise...
rouge.get_scores(model_in, refr, avg=True)#recover and print scores
